## 1. Environment & Requirements
This notebook will:
1. Clone the GitHub repository (or you can upload a zip).
2. Install Python dependencies.
3. Allow secure upload of required secret/config files (`oci_api_key.pem`, `.env`, `config.properties`, `api_spec_*.json`).
4. Launch AskMeChatBot Flask app (port 5000).
5. Start a Cloudflare tunnel and display the public URL.
6. Test /chat endpoint via the tunnel.
Latency note: Gen AI calls target `us-ashburn-1` from Colab region. You may adjust `GENAI_REGION`.

In [ ]:
# 2. Runtime diagnostics
import sys, platform, subprocess
print('Python:', sys.version)
print('Platform:', platform.platform())

In [ ]:
# 3. Clone repository (public). If already cloned, skip.
import os
REPO_URL = 'https://github.com/Ramsiit2010/OCIGenAIBot.git'
TARGET_DIR = 'OCIGenAIBot'
if not os.path.isdir(TARGET_DIR):
    !git clone $REPO_URL
else:
    print('Repository already present.')

In [ ]:
# 4. Install dependencies
%cd OCIGenAIBot
!pip install --quiet --upgrade pip
!pip install --quiet -r requirements.txt
print('Dependencies installed.')

## 5. Upload Secrets & Config Files
Upload any of these files as needed: 
- `oci_api_key.pem` (for OCI user principal)
- `.env` (OCI credentials & GENAI settings)
- `config.properties` (backend endpoints)
- `api_spec_*.json` (if overriding)
Skip if already baked into repo or using Resource Principal (not applicable here).

In [ ]:
from google.colab import files
print('Select files to upload (optional).')
uploaded = files.upload()
for name, data in uploaded.items():
    with open(name, 'wb') as f: f.write(data)
print('Uploaded files:', list(uploaded.keys()))

## 6. Fallback .env Creation (Optional)
If you did not upload a .env file, create a minimal one below (user principal).

In [ ]:
import os
if not os.path.isfile('.env'):
    with open('.env','w') as f:
        f.write(
'''OCI_USER=ocid1.user.oc1..xxxx
OCI_TENANCY=ocid1.tenancy.oc1..xxxx
OCI_FINGERPRINT=aa:bb:cc:dd:ee:ff:00:11:22:33:44:55:66:77:88:99
OCI_KEY_FILE=oci_api_key.pem
OCI_REGION=us-ashburn-1
GENAI_REGION=us-ashburn-1
GENAI_MODEL=cohere.command-plus-latest
GENAI_INTENT_MODE=auto
'''
        )
    print('.env created placeholder. Update values if needed.')
else:
    print('.env already exists, not overwriting.')

## 7. Install Cloudflared (Cloudflare Tunnel Client)

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb || sudo apt-get -f install -y
!cloudflared --version

## 8. Launch AskMeChatBot (Background)

In [ ]:
import subprocess, time
askme_proc = subprocess.Popen(['python','AskMeChatBot.py'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
time.sleep(5)
print('AskMeChatBot started (port 5000).')

## 9. Start Cloudflare Tunnel and Extract Public URL
Tunnel will bind to http://localhost:5000 and output a trycloudflare.com URL.

In [ ]:
import re, time, threading
tunnel_log = 'cloudflared.log'
tunnel_proc = subprocess.Popen(['cloudflared','tunnel','--url','http://localhost:5000','--no-autoupdate'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
public_url = None
for _ in range(60):
    line = tunnel_proc.stdout.readline()
    if not line: time.sleep(1)
    else:
        print(line.strip())
        m = re.search(r'https://[-a-z0-9]+trycloudflarecom', line)
        if m:
            public_url = m.group(0)
            break
print('
Public Tunnel URL:', public_url or 'NOT FOUND (check logs)')

## 10. Test /chat Endpoint via Tunnel

In [ ]:
import requests, json
assert public_url, 'Tunnel URL not found.'
payload = {'prompt':'List recent orders'}
r = requests.post(public_url.rstrip('/') + '/chat', json=payload, timeout=30)
print('Status:', r.status_code)
print('Response JSON:', r.json())

## 11. View Live Logs (Optional)

In [ ]:
# Non-blocking sample of recent output
import itertools
print('--- AskMeChatBot (first 40 lines) ---')
for i, line in zip(range(40), askme_proc.stdout):
    print(line.decode().rstrip())
print('--- End ---')

## 12. Cleanup (Stop Processes)

In [ ]:
# Run this if you want to terminate the background processes
try:
    askme_proc.terminate()
    tunnel_proc.terminate()
    print('Processes terminated.')
except Exception as e:
    print('Cleanup error:', e)